# Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models, layers
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import glob

# Load the dataset and Normalize

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


# Expand dimensions for CNN input

In [ ]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Create a Model CNN

In [ ]:

model = models.Sequential([
    layers.Input(shape=(28, 28, 1)),

    layers.Conv2D(32, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 11, 11, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       409,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 432,650 (1.65 MB)

 Trainable params: 431,946 (1.65 MB)

 Non-trainable params: 704 (2.75 KB)

# Train the Model

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 84s 95ms/step - accuracy: 0.8826 - loss: 0.3931 - val_accuracy: 0.9842 - val_loss: 0.0561
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 85s 98ms/step - accuracy: 0.9744 - loss: 0.0823 - val_accuracy: 0.9857 - val_loss: 0.0469
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 139s 95ms/step - accuracy: 0.9801 - loss: 0.0659 - val_accuracy: 0.9897 - val_loss: 0.0348
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 83s 96ms/step - accuracy: 0.9846 - loss: 0.0492 - val_accuracy: 0.9913 - val_loss: 0.0302
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 84s 99ms/step - accuracy: 0.9852 - loss: 0.0473 - val_accuracy: 0.9903 - val_loss: 0.0373


#save The Model


In [ ]:
model.save("digit_reconoizer.h5")

#Image preprocessing

In [ ]:
model = load_model("digit_reconoizer.h5")


# function to covert the image to fit the model to recogonizes it
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')
    img = ImageOps.invert(img)
    img = img.resize((28, 28))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=(0, -1))
    return img


# the np values to given to the model
imgs = glob.glob("./*.png")


for image in imgs:
  img = preprocess_image(image)

  # predicted values from 1 - 9 with probability of values

  predicted = model.predict(img)

  # return the idx with heighest probablilty

  number_predicted = np.argmax(predicted)

  print(f"The predicted number is for {image}: {number_predicted}")




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
The predicted number is for ./Screenshot 2025-05-04 204848.png: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
The predicted number is for ./Screenshot 2025-05-04 205038.png: 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
The predicted number is for ./Screenshot 2025-05-04 204817.png: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
The predicted number is for ./Screenshot 2025-05-04 205000.png: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
The predicted number is for ./Screenshot 2025-05-04 204305.png: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
The predicted number is for ./Screenshot 2025-05-04 205055.png: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
The predicted number is for ./Screenshot 2025-05-04 205018.png: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
The predicted number is for ./Screenshot 2025-05-04 204943.png: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
The predicted number is for ./Screenshot 2025-05-04 204906.png: 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
The predicted number is f